# Assignment Applied Machine Learning BSc IK 

## Notebook made by

**Gebruik graag dit formaat**

* Voor de namen:  voornaam rest van je naam, voornaam rest van je naam,....
* je studentnummers: hetzelfde: scheidt met `,`
* je emails: hetzelfde: scheidt met `,`
* voor je groep: **alleen de hoofdletter** (iets als  `A` of `B` dus)

__Namen__:Anoniem
__Emails__:Anoniem
__Student id__:Anoniem
__Groep__:Anoniem

## Toelichting

* Een aantal opgaven worden automatisch nagekeken. Bij vrijwel alle opdrachten staan er een paar tests onder de opdracht, dit is voornamelijk om te zorgen dat je de juiste type output geeft. Dit zijn dus *NIET* alle tests, die komen er bij het graden nog bij.
* Elke vraag is 1 punt waard, tenzij anders aangegeven. Soms is die punt onderverdeeld in deelpunten, maar niet altijd. 

## Voor het inleveren!

* Pas niet de cellen aan, vooral niet die je niet kunt editen. Dit levert problemen op bij nakijken. Twijfel je of je per ongeluk iets hebt gewijzigd, kopieer dan bij inleveren je antwoorden naar een nieuw bestand, zodat het niet fout kan gaan.

* Zorg dat de code goed runt van boven naar beneden, verifieer dat door boven in Kernel -> Restart & Run All uit te voeren

## Na het inleveren!

* Het gebeurt erg vaak dat mensen een "leeg bestand" inleveren. Vaak een andere versie van de opgave die nog ergens op je computer rondslingerde. Zonde van al je werk toch!
* Dus, lever **minstens een half uur voor tijd in**. Download dan wat je hebt ingeleverd op Canvas. Geef het een andere naam om verwarring te voorkomen. En draai alle cellen, en bekijk het. Geen syntax fouten? Alle vragen gemaakt? Dan zit het vast wel goed, en hoef je niet in de zenuwen te zitten.

# Applied Machine Learning W2

This assignment tests the concepts that have been discussed in this week's lecture, and is divided in several (mostly) independent exercises. 

1. [Agglomerative Clustering](#agglo)
2. [DBSCAN](#dbscan)
3. [Evaluation](#elm)
4. [Data Normalization](#normalization)
5. [PCA](#pca)

### TIP
The second assignment, which asks you to implement the DBSCAN algorithm, can be a bit challenging, so don't forget about the other exercises! Just start working on another part of the assignment if you get stuck, and come back later.

In [ ]:
%matplotlib inline

import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from nose.tools import assert_count_equal, assert_equal
from numpy.testing import *
from sklearn import datasets
# Please do not remove this: 
np.random.seed(31415)

<a id="agglo">

# 1 Agglomerative Clustering

1. We defined the Ward criterion in class. Now you define from scratch, similar as we did with Ward, *average* and *complete*.
2. Create the complete algrithm for bottom up agglomerative clustering from scratch, giving, just like in sklearn, the possibility to choose the linking function. We will help you by having you implement the parts needed as seperate functions.
3. Use the Euclidean distance in your functions.
3. Try out your code on the iris dataset.
4. Compare it to sklearn. I mean the outcomes on the same input. 
5. Use the technique described in the book to create a dendogram.

## Tips
* explain what you are doing
* test and test and test
* refactor, refactor
* Be proud. Don't despair if it takes hours to write a few lines of code.
    * That is because Python is such a fantastic high level language.

In [ ]:
# Start by completing the two functions below, which both takes matrices of size n_samples*n_features
# and return the distance between those clusters according to the linkage functions
def average_linkage(cluster1, cluster2) -> np.float64:
    '''calculates the average distance between cluster 1 and cluster 2'''
    # the distance between cluster1 and cluster 2 = the Squared Euclidian distance between the centers of cluster1 and cluster 2.
    distance = sum((cluster1.mean(axis=0) - cluster2.mean(axis=0)) ** 2)
    return distance

def complete_linkage(cluster1, cluster2) -> np.float64:
    '''calculates maximum distance between cluster 1 and cluster 2'''
    # the max distance between cluster1 and cluster 2 = the Squared Euclidian distance between the min-value of cluster1 and max-value of cluster 2.
    distance = sum((cluster1.min(axis=0) - cluster2.max(axis=0)) ** 2)
    return distance



In [ ]:
# check that you get the right output
cluster_1 = np.random.rand(5, 2)
cluster_2 = np.random.rand(10, 2)
assert_equal(type(average_linkage(cluster_1, cluster_2)), np.float64)

After implementing the two linkage functions, you will implement a single step of the clustering algorithm, which will return the IDs of the clusters in the input that should be merged. As input you will have both the original input data, as well as a dictionary of clusters, where each key is an ID, and each value is a list of points belonging to that cluster. You can use fancy indexing from numpy to get the rows belonging to a cluster easily:

```
# suppose we have input data X
clusters = {0: [0, 1, 2, 3], 4: [4, 6, 7]}
# get the vectors of our first cluster
cluster_vectors = X[clusters[0]]
```

In [ ]:
def clustering_step(input_data, cluster_dict, linkage_type: str) -> tuple:
    ''' Function that, given an input of vectors such as in the hoorcollege slides
    {cluster_id: matrix}
    returns the SORTED IDs of the two clusters that should be merged as a list'''
    assert linkage_type in ['average', 'complete']
    
    #WRITE YOUR CODE HERE
    if linkage_type == 'average':
        distance = {(i, j): average_linkage(input_data[[i]], input_data[[j]]) for i in cluster_dict for j in cluster_dict if i != j}
        cluster_ids = sorted(distance)[:1][0]
    elif linkage_type == 'complete':
        distance = {(i, j): complete_linkage(input_data[[i]], input_data[[j]]) for i in cluster_dict for j in cluster_dict}
        cluster_ids = sorted(distance)[:1][0]
           
    return tuple(sorted(cluster_ids))


In [ ]:
input_array = np.array([
    [0, 0, 0],
    [1, 1, 1],
    [3, 3, 3]])
clusters = {0: [0], 1: [1], 2: [2]}

assert_equal(type(clustering_step(input_array, clusters, linkage_type='average')), tuple)

In our last step, we will implement the complete algorithm for agglomerative clustering, using our previously created `clustering_step` and linkage functions. For the return value, return a list of labels where each label corresponds to the class of the instance. It doesn't matter if these values are not 0/1 as long as items in the same cluster share the same label. 

**TIP**
You can delete key value pairs from dictionaries using the `del dictionary[key]` syntax

In [ ]:
# in our version, we will define a number of clusters we want to find
# you may also define this in sklearn for your comparison later
def agglomerative_clustering(input_points, number_of_clusters: int, linkage_type: str):
    assert linkage_type in ['average', 'complete']
    
    # We start by just having each point in its own cluster
    # update this dictionary in the while loop.

    clusters = {i: [i] for i in range(len(input_points))}
    
    for i in range(len(input_points)):
        clusters[i] = input_points[i] 
    print(clusters)
    # Look at the stopping condition, understand why?
    while len(clusters) != number_of_clusters:
        #WRITE YOUR CODE HERE
        a = len(clusters) + 1
        clusters.update({a:clustering_step(input_points, clusters, linkage_type)})

        del clusters[clusters[a][0]]
        del clusters[clusters[a][1]]
        
        
    # Your output should be a dict with as keys the resulting cluster IDs and as values the points belonging to
    # that cluster. It does not matter what the ID itself is.
    
    # create the output array (we will do this for you)
#     output_labels = np.zeros((input_points.shape[0], 1))
#     for i, point_ids in enumerate(clusters.values()):
#         output_labels[point_ids] = i
    
    
    return output_points

samples = np.random.rand(10, 3)
agglomerative_clustering(samples, number_of_clusters=4, linkage_type="average")

In [ ]:
# Did we actually create the right amount of clusters?
samples = np.random.rand(10, 3)
labels = agglomerative_clustering(samples, number_of_clusters=4, linkage_type="average")
assert_equal(len(np.unique(labels)), 4)

In [ ]:
# Compare your output with sklearn here
from sklearn.cluster import AgglomerativeClustering

#WRITE YOUR CODE HERE

Now that you have created the agglomerative algorithm, create the dendogram and plot it.

In [ ]:
#WRITE YOUR CODE HERE

<a id="dbscan" />

# 2 DBscan

In class and in section of the book 3.5.3 the DBScan algorithm was discussed.
In this assignment we are going to implement the DBScan metric from scratch, working from first principles and using euclidean distance as the distance metric.

**TIPS**
- Implementing this from scratch is a challenge, but it should be doable, make sure you really understand/can explain the algorithm before you start coding, this should already help you when programming. Like before, we will also make you implement sub functions first to help you along.
- Remember that in the algorithm we include the sample itself in the calculation of the distances and `min_samples`, you don't have to write separate logic to exlcude the point itself from the neighbour calculations.

In [ ]:
def get_neighbours(input_data, point_index, eps) -> list:
    """
    Given the entire input data and the index of a specific point, get the indices of all
    the neighbor points, i.e. points that are closer than eps away using euclidean distance.
    """
    #WRITE YOUR CODE HERE    
    
    return [i for i in range(input_data.shape[0]) if euclidean_distances(input_data[[point_index]], input_data[[i]]) <= eps]


In [ ]:
# check that the output contains integers, which should correspond
# to the indices of the neighbours
test_data = np.random.rand(5, 3)
assert_equal(type(get_neighbours(test_data, 0, eps=1)[0]), int)

After we have implemented the function to get all neighbours of a point for a specific point, we will write a function that, starting from a specific point, finds all neighbour points, and their neighbours points, and so on, until the cluster can no longer be expanded.

## TIPS
 - In this function and the next, we will assume points that have a value of `-1` in the cluster are ure (currently) outliers, and points with a value of `-2` have not yet been examined.

In [ ]:
def expand(input_data, clusters, point, class_label, eps, min_samples):
    """
    Given the input data, the array with the current clusters assignments,
    and the ID for the class, return all the points in the class, iteratively
    going over neighbours, neigbours of neighbours etc.
    """
    neighbours = get_neighbours(input_data, point, eps)
    clusters[point] = class_label
    
    i = 0
    while i < len(neighbours):
        p = neighbours[i]
        if clusters[p] == -1:
            clusters[p] = class_label
        elif clusters[p] == 0:
            clusters[p] = class_label
            p_neighbours = get_neighbours(input_data, p, eps)
            if len(p_neighbours) >= min_samples:
                neighbours += p_neighbours
        i += 1
        
    return clusters

In the cell below we will put everything we made so far together and create the complete DBSCAN algorithm. 


In [ ]:
def dbscan(input_data, eps, min_samples):
    """
    Implements the complete dbscan algorithm using the helper functions we wrote before.
    Takes as input the complete input matrix, the maximum distance neighbours can be apart, eps,
    and the minimal number of samples that are neighbours from a point to let it be considered
    a core point. Returns an array with for each item the class label of that item, where -1
    indicates that the item is an outlier.
    """
    current_class = 0
    # start with empty cluster assignments
    clusters = [-2 for _ in range(input_data.shape[0])]
    
    for i in range(0, input_data.shape[0]):
        if clusters[i] == -2: # unvisited
            neighbours = get_neighbours(input_data, i, eps)
            if len(neighbours) < min_samples: # not a core point or border point
                clusters[i] = -1 # outlier
            else:
                current_class += 1 # create a new class
                clusters = expand(input_data, clusters, i, current_class, eps, min_samples)
                
    return np.array(clusters)

Compare the output of your algorithm to the output of sklearns `DBSCAN` in the cell below, explaing your tests in the comments of the cell

### TIP

- Have a look at `make_blobs` from the sklearn.datasets module, this is a nice function with which you can easily
create some sample clusters to test your algorithm.

In [ ]:
# Compare to sklearn
from sklearn.cluster import DBSCAN
#WRITE YOUR CODE HERE

<a id="elm" />

# 3 Evaluation

We can evaluate a cluster algorithm against a gold standard labelling. The tricky part then is which cluster to assign to which label. Think of Iris, and you have clustered it into 3 groups. Then which group do you want to assign to which species?

A smart idea called BCubed, and later improved to [ELM](https://dl.acm.org/doi/abs/10.1145/3539813.3545121) can be used to evaluate the quality of a clustering.

* implement ELM
* For this exercise, implement the F1 score, where you calculate F1 for each element and then average it over all elements
* test it with a number of clusterings you made of the IRIs set and another own found set with gold standard.
 - If you managed to implement your own version DBSCAN you can use these to make clusters of the IRIS dataset, otherwise just use DBSCAN from `sklearn`.
* Experiment with normalization, and see the effect.
* You are not training anything, so you simply cluster the full set and use the same set as test set.

* Next to the code, you write a small report in a markdown cell on your findings.

**TIPS**:
 - The ELM algorithm operates on points, and compares the cluster a point is in in the
 gold standard with a cluster a point is in in the prediction. 
 - Using sets is useful!
 - Start by creating a nested dict where each key is the index of a point, and each value is a dict with the indices of its neighbours, exluding the element itself. Do this for buth the gold standard and the predicted cluster.


### Test dbscan's parameters

* Using our ELM algorithm, try several options for the DBSCAN algorithm and report which values worked best.

In [ ]:
# your input will be an array of point labels, as you would get as the output of an sklearn
# model.
def ELM(gold_standard_labels, predicted_labels):
    score = []
    num_points = len(gold_standard_labels)
    #WRITE YOUR CODE HERE
      
    return score

In [ ]:
assert_equal(type(ELM([0, 1, 2, 2], [0, 0, 0, 0])), np.float64)

In [ ]:
# Try different variations of DBSCAN, and evaluate using the ELM score we just defined.


#WRITE YOUR CODE HERE

<a id="normalization" />

# Preprocessing and scaling section 3.2


1. Download iris data, and Z-transform it using your own code.
    2. Give a convincing check/test(s) that it worked.
2. Similary for the `min-max` scalar and sklearn's `normalizer`.
    * Of course you also describe in good English and with a math formula what this scaler is doing.
3. Figure out how sklearn's *RobustScaler* works, and implement it using your own code. You can use numpy to get the 1st and 3rd quartile.
4. Test all your code on iris and at least one other dataset.
    * Make your own tests which really test the math.
    * Run the same ytransformation in sklearn and use something like `assert_array_almost_equal` to test that you are close enough to sklearn's.

Answer both in code and in a markdown cell.

In [ ]:
# get the iris dataset
iris_X, iris_y = sklearn.datasets.load_iris(return_X_y=True)

def z_transform(input_data):
    output_data = []
    #WRITE YOUR CODE HERE
    return np.vstack(output_data)

def my_normalize(input_data):
    output_data = []
    #WRITE YOUR CODE HERE
    return np.vstack(output_data)
    

def min_max(input_data, min_val, max_val):
    output_data = []
    #WRITE YOUR CODE HERE
    return np.vstack(output_data)

def robust_scaler(input_data):
    output_data = []
    #WRITE YOUR CODE HERE
    return np.vstack(output_data)
    


Briefly describe for each scaler what they do (and give the formula)

YOUR ANSWER HERE

For each of the functions you implemented above, write tests in the cell below to make sure your scaling is correct

In [ ]:
#WRITE YOUR CODE HERE

<a id="pca" />

# PCA

### inspect
* get the sklearn digit dataset.
* transform it to 2 dimensions using PCA, you are allowed to use the `sklearn` function for this exercise.
* plot it in 2 dimensions using seaborn and use the `hue` parameter to see how much the digits are separated.
* Does it make sense? Do you see similar "confusions/overlap" as seen before in the book between certain digits?
 - Play around with the color palete to ensure that the different classes have clearly different colours.


### cluster

Run K-means on this reduced data (2 instead of 64 dimensions!), also run K-means on the original data. Then compare the two using the ELM-metric. For this experiment you are allowed to use the K means algorithm from sklearn.

In [ ]:
from sklearn.decomposition import PCA
# Load in the digit dataset
digits_X, digits_y = sklearn.datasets.load_digits(return_X_y=True)
# You are allowed to fix the pca to always return two dimensions
def perform_pca(input_vectors):
    output_vectors = None
    
    #WRITE YOUR CODE HERE

    return output_vectors

In [ ]:
assert_equal(perform_pca(digits_X).shape[-1], 2)

In [ ]:
# Implement your plotting here
import matplotlib.pyplot as plt
#WRITE YOUR CODE HERE

Does your plot make sense? Do you see similar overlaps as in the book?

YOUR ANSWER HERE

In [ ]:
# Compare K means on both the reduced and unreduced data and evaluation using ELM.
from sklearn.cluster import KMeans

#WRITE YOUR CODE HERE